In [3]:
!pip install brian2
!pip install -q condacolab
import condacolab
condacolab.install()
!conda create -y -n opensim_env python=3.10
!source /usr/local/etc/profile.d/conda.sh && conda activate opensim_env && conda install -y -c opensim-org opensim
!unzip Model.zip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.7 MB/s eta 0:00:00
⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...
Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/opensim_env

  added / updated specs:
    - python=3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.1.31  |       hbcca054_0         154 KB  conda-forge
    ld_impl_linux-64-2.43      |       h712a8e2_4         656 K

In [ ]:
from brian2 import *
import numpy as np
import pandas as pd
import os
import subprocess
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

In [ ]:
t_stretching_afferent = 16*ms
t_motoneuron_muscle = 10*ms
t_reaction = t_stretching_afferent + t_motoneuron_muscle
n_loop = 10
dt = 0.1*ms
name_output = 'time_series.csv'
n = {"Ia": 10, "II": 10, "exc": 10, "motor": 10}

# Initialize stretch and velocity arrays
stretch_init = np.zeros(int(t_reaction/dt))
velocity_init = np.zeros(int(t_reaction/dt))
stretch = stretch_init
velocity = velocity_init

data = []
for j in range(1, n_loop):
    time = np.arange(j*t_reaction, (j+1)*t_reaction, dt)

    # Run neural simulation
    subprocess.run([
        "python", "neurons_simulation.py",
        "--stretch", stretch_init,
        "--velocity", velocity_init,
        "--neuron_population", n,
        "--dt", str(dt),
        "--T", str(t_reaction)
    ])

    # Load spike data (fixed syntax)
    spikes_times_Ia = np.load("spikes_Ia.npy")
    spikes_times_II = np.load("spikes_II.npy")
    spikes_times_motor = np.load("spikes_motor.npy")

    # Decode spikes to activation
    os.system("python decode_spikes_to_activation.py --spikes_times_motor spikes_motor.npy")
    activations = np.load("activation.npy")
    mean_activation = np.mean(activations, axis=0)

    # Run muscle simulation
    os.system(f"source /usr/local/etc/profile.d/conda.sh && conda activate opensim_env && python muscle_sim.py --time {time} --activation {mean_activation}")
    table = osim.TimeSeriesTable("muscle_state.sto")

    # Create batch data dictionary
    batch_data = {
        'time': time,
        **{f'spikes_times_II_{i}': time == spikes_times_II[i] for i in range(n['II'])},
        **{f'spikes_times_Ia_{i}': time == spikes_times_Ia[i] for i in range(n['Ia'])},
        **{f'spikes_times_motor_{i}': time == spikes_times_motor[i] for i in range(n['motor'])},
        **{f'activation_motor_{i}': activations[i] for i in range(n['motor'])},
        'mean_activation': mean_activation,  # Fixed variable name
        'stretch': table.getDependentColumn('stretch').to_numpy(),  # Fixed table access
        'velocity': table.getDependentColumn('velocity').to_numpy(),
        'fiber_length': table.getDependentColumn('fiber_length').to_numpy(),
        'fiber_velocity': table.getDependentColumn('fiber_velocity').to_numpy()
    }

    data.append(batch_data)

# Create DataFrame and apply time shifts
df = pd.DataFrame(data)
time_shift = int(t_reaction/dt)
df['time'] = df['time'].shift(periods=time_shift).fillna(method='bfill')  # Fixed shift operation

# Select columns that contain spike times
spike_columns = [col for col in df.columns if 'spikes_times' in col]
time_shift_spikes = int(t_stretching_afferent/dt)

# Shift each of these columns by t_stretching_afferent
for col in spike_columns:
    df[col] = df[col].shift(periods=time_shift_spikes)

# Select the remaining columns
others_columns = [col for col in df.columns if col not in spike_columns]
for col in others_columns:
    df[col] = df[col].shift(periods=time_shift)  # Fixed parenthesis

# Set initial values
df.loc[:time_shift, ['stretch', 'velocity']] = [stretch_init, velocity_init]  # Fixed loc syntax

# Save the DataFrame to CSV
df.to_csv(name_output, mode='a', header=False, index=False)

# Plot time series
plot_times_series(name_output)

TypeError: expected str, bytes or os.PathLike object, not ndarray